In [1]:
import os
from random import randint
from random import shuffle
from guizero import App, Box, Picture, warn, Text, TextBox, PushButton, info, Window

emojis_dir='/home/User/Downloads/emojis' #Download the emoji folder provided and set the pathname.
app = App("Emoji Match")
intro_game = Text(app, text="Welcome to the Emoji Match where you can match emoji's.\n\nThis is a two player game.\n\n Please enter the names of the players")
eb1 = Text(app, text="")
player_box = Box(app, layout="grid")
frst_player = Text(player_box, text="First Player", grid=[0,0])
txt_frst = TextBox(player_box, grid=[1,0])
eb2 = Text(player_box, grid=[0,1])
scnd_player = Text(player_box, text="Second Player", grid=[0,2])
txt_scnd = TextBox(player_box, grid=[1,2])
eb3 = Text(player_box, grid=[0,2])
eb4 = Text(player_box, grid=[0,3])
plyr = txt_frst.value
frst_score = 0
scnd_score = 0
bonus = 0
val = 0

def game_rules():
    instr = Window(app, title="Instructions for the game", width = 800)
    rules = Text(instr, text="1. The first player plays first.\r2. When time runs out, the player\'s turn ends and the next player\'s turn begins.\r3. For each correct match, the player gets one point.\r4. For 3 consecutive correct matches, the player gets a bonus point.\r5. For each wrong match, the player loses one point.")
    instr.show()
    
inst = PushButton(player_box, text="Instructions", command=game_rules, grid=[0,4])

def start_game():
    global plyr, curr_plyr
    window = Window(app, title="Emoji Match", height=600, width=500)
    pic_box = Box(window, layout="grid")
    button_box = Box(window, layout="grid")
    extra_features = Box(window)
    timer = Text(extra_features, text="Start")
    buttons = []
    pics = []
    result = Text(window)
    plyr_name = Box(window)
    score_board = Box(window)

    curr_plyr = Text(plyr_name, text=str(txt_frst.value), align="left")
    label = Text(score_board, text="Score", align="left")
    score = Text(score_board, text="0", align="left")
    bonus = 0
    for x in range(0,3):
        for y in range(0,3):
            button = PushButton(button_box, grid=[x,y])
            buttons.append(button)
            picture = Picture(pic_box, grid=[x,y])
            pics.append(picture)
            
    def emoji_match(matched):
        global bonus, val
        if matched:
            result.value = "Correct"
            score.value = int(score.value) + 1
            val += 1
            bonus += 1
            if bonus == 3:
                timer.cancel(counter)
                info("Bonus Point", "Congratulations! You've earned an extra point.")
                score.value = int(score.value) + 1
                val += 1
                bonus = 0
                timer.value = "20"
            else:
                timer.cancel(counter)
                timer.value = "20"

            setup_round()
            timer.repeat(1000, counter)
        else:
            result.value = "Incorrect"
            score.value = int(score.value) - 1
            val = val -1
            bonus = 0
        
    def counter():
        global bonus, curr_plyr, frst_score, scnd_score, txt_frst, txt_scnd, val
        timer.value = int(timer.value) - 1
        if int(timer.value) == 0:
            global bonus, frst_score, scnd_score
            timer.cancel(counter)
            result.value = "Game Over"
            if curr_plyr.value == txt_frst.value:
                curr_plyr.value = txt_scnd.value
                bonus = 0
                frst_score = frst_score + int(score.value)
            else:
                curr_plyr.value = txt_frst.value
                bonus = 0
                scnd_score = scnd_score + int(score.value)
            
            warn("Time Out", "You've run out to time. "+str(curr_plyr.value)+" get ready to play!")
            timer.value = "20"
            score.value = "0"
            val = 0
            result.value = ""
            setup_round()
            timer.repeat(1000, counter)
        
    def setup_round():
        emojis = [os.path.join(emojis_dir, f) for f in os.listdir(emojis_dir) 
             if os.path.isfile(os.path.join(emojis_dir, f))]
        shuffle(emojis)
        matched_emoji = emojis.pop()

        for picture in pics:
            picture.image = emojis.pop() 
        
        rand_pic = randint(0,8)
        pics[rand_pic].image = matched_emoji
        
        for button in buttons:
            button.image = emojis.pop()
            button.update_command(emoji_match, [False])
        
        rand_button = randint(0,8)
        buttons[rand_button].image = matched_emoji
        buttons[rand_button].update_command(emoji_match, [True])
        
    def close_window():
        timer.cancel(counter)
        bonus=0
        window.destroy()
    
    setup_round()
    timer.value = 20
    timer.repeat(1000, counter)
    window.when_closed = close_window

def end_game():
    global curr_plyr, txt_frst, txt_scnd, frst_score, scnd_score, val
    if curr_plyr.value == txt_frst.value:
        frst_score = frst_score + val
    else:
        scnd_score = scnd_score + val
    if frst_score > scnd_score:
        app.info("Results", "The winner of this game is "+str(txt_frst.value)+"!\rCongratulations "+str(txt_frst.value)+".\r Player        Score \r"+str(txt_frst.value)+": "+str(frst_score)+"\r"+str(txt_scnd.value)+": "+str(scnd_score))
    elif scnd_score > frst_score:
        app.info("Results", "The winner of this game is "+str(txt_scnd.value)+"!\rCongratulations "+str(txt_scnd.value)+".\r Player        Score \r"+str(txt_frst.value)+": "+str(frst_score)+"\r"+str(txt_scnd.value)+": "+str(scnd_score))
    else:
        app.info("Results", "It's a tie!\r Player        Score\r"+str(txt_frst.value)+": "+str(frst_score)+"\r"+str(txt_scnd.value)+": "+str(scnd_score))
    app.destroy()
    
end = PushButton(app, text="End", command=end_game, align="bottom")
ep5 = Text(app, text="", align="bottom")
ep6 = Text(app, text="", align="bottom")
ep7 = Text(app, text="", align="bottom")
ep8 = Text(app, text="", align="bottom")
ep9 = Text(app, text="", align="bottom")
start = PushButton(app, text="Start", command=start_game, align="bottom")
app.when_closed = end_game
app.display()
